# Sort Query Results
#### (WDS objects have been queried in Gaia, now it's time to sort them based on parallax, separation, and proper motion)
#### These methods origniated in the el Badry et al publication in 2021 - implemented by Daphne Zakarian

In [1]:
from astropy.io import ascii
from astropy.table import vstack, Table, unique, QTable
from astropy.coordinates import SkyCoord 
import astropy.units as u
from astropy import table, log
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Distance, Angle
from astropy.time import Time

import os
from IPython.display import display
from multiprocessing import set_start_method

from math import sqrt, cos

## Read in the stacked results table and assign units to the columns

In [2]:
# google colab read file in: 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# after mounting drive, copy the path of the directory where the files are stored
save_path='/content/drive/MyDrive/NOFS copy/WiserLabBackup/QueryResults'

In [5]:
## Read the file

## vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# # wiser's lab comp:
# save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

qrt ='{0}/stack_query_results_table.ecsv'.format(save_path) 
ie = '{0}/stack_index_error_queries.ecsv'.format(save_path)
query_results_table = QTable.read(qrt, header_start=0, data_start=1)
index_error_queries = Table.read(ie, header_start=0, data_start=1)


## Assign Units: 

## DEGREES 

deg_columns = ['ra_a', 'ra_b', 'dec_a', 'dec_b']
for deg_column in deg_columns:
    query_results_table[deg_column] = query_results_table[deg_column]*u.deg
    
## MAS

mas_columns = ['ra_error_a', 'ra_error_b', 'dec_error_a', 'dec_error_b', 'parallax_a', 'parallax_b', 'parallax_error_a', 'parallax_error_b']
for mas_column in mas_columns:
    query_results_table[mas_column] = query_results_table[mas_column]*u.mas

## MAS/YR
mas_p_year_columns = ['pmra_a', 'pmra_b', 'pmra_error_a', 'pmra_error_b', 'pmdec_a', 'pmdec_b', 'pmdec_error_a', 'pmdec_error_b']
for mas_p_year_column in mas_p_year_columns:
    query_results_table[mas_p_year_column] = query_results_table[mas_p_year_column]*u.mas/u.yr

In [14]:
query_results_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292,6.593528748410601,5.849294523358041,65.37566605693283,0.029512516295242767,8.08734681915077,0.032454123037433946
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429,8.13322312188097,22.71564110834343,1.178308274619283,0.05300534921025606,1.1096457199617988,0.0686100192383481
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.000

In [7]:
# add calculated columns

# SEPARATION

# the mu columns are all proper motion calculations
query_results_table.add_column(0.0*u.arcsec, name='separation')
query_results_table.add_column(0.0*u.mas/u.yr, name='delta_mu_orbit')
query_results_table.add_column(0.0*u.mas**2/u.yr**2, name='delta_mu_ra2')
query_results_table.add_column(0.0*u.mas**2/u.yr**2, name='delta_mu_dec2')
query_results_table.add_column(0.0*u.mas/u.yr, name='delta_mu')
query_results_table.add_column(0.0*u.mas/u.yr, name='sigma_delta_mu')


for rownum in range(len(query_results_table)):
    ra_a, ra_b = query_results_table[rownum]['ra_a'], query_results_table[rownum]['ra_b']
    dec_a, dec_b = query_results_table[rownum]['dec_a'], query_results_table[rownum]['dec_b']
    # use the ra and dec of component a and b to make SkyCoord objects
    coord_a = SkyCoord(ra_a, dec_a, frame='icrs')
    coord_b = SkyCoord(ra_b, dec_b, frame='icrs')

    # calcualte the separation between the two objects
    separation = coord_a.separation(coord_b).to(u.arcsec)

    query_results_table['separation'][rownum] = separation
    if rownum%5000==0:
        print(rownum)

 

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [ ]:
query_results_table

## Create output tables: associated systems and non-associated systems
#### I can't find a simple way to copy astropy tables so I'll just reopen the query results table and assign units like before
#### I think assigning a new name still refers to the og table... whatever was causing the issue, this way works fine!!


I will start with the associated systems table being identical to the query_results_table.
As I loop the results table, the row number of the non-associated systems will be appended to a list.
After the loop, I will use these row numbers to copy the non-associated systems to their output table, 
and also to remove those rows from the associated systems table.

In [8]:
# Read the file

# ## vayu's lab comp:
# # save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# # wiser's lab comp:
# save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

# qrt ='{0}/stack_query_results_table.ecsv'.format(save_path) 


associated_systems_table = QTable.read(qrt, header_start=0, data_start=1)
non_associated_systems_table = QTable.read(qrt, header_start=0, data_start=1)








## Assign Units: 

## DEGREES 

deg_columns = ['ra_a', 'ra_b', 'dec_a', 'dec_b']
for deg_column in deg_columns:
    associated_systems_table[deg_column] = associated_systems_table[deg_column]*u.deg
    non_associated_systems_table[deg_column] = non_associated_systems_table[deg_column]*u.deg
## MAS

mas_columns = ['ra_error_a', 'ra_error_b', 'dec_error_a', 'dec_error_b', 'parallax_a', 'parallax_b', 'parallax_error_a', 'parallax_error_b']
for mas_column in mas_columns:
    associated_systems_table[mas_column] = associated_systems_table[mas_column]*u.mas
    non_associated_systems_table[mas_column] = non_associated_systems_table[mas_column]*u.mas
## MAS/YR
mas_p_year_columns = ['pmra_a', 'pmra_b', 'pmra_error_a', 'pmra_error_b', 'pmdec_a', 'pmdec_b', 'pmdec_error_a', 'pmdec_error_b']
for mas_p_year_column in mas_p_year_columns:
    associated_systems_table[mas_p_year_column] = associated_systems_table[mas_p_year_column]*u.mas/u.yr
    non_associated_systems_table[mas_p_year_column] = non_associated_systems_table[mas_p_year_column]*u.mas/u.yr
    

# add calculated columns
# separation calcualted ahead of time, proper motion info (all of the mu and sigmas) are calculated in the loop


associated_systems_table.add_column(0*u.arcsec, name='separation')
associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu_orbit')
associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_ra2')
associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_dec2')
associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu')
associated_systems_table.add_column(0*u.mas/u.yr, name='sigma_delta_mu')

# for rownum in range(len(associated_systems_table)):
#     ra_a, ra_b = associated_systems_table[rownum]['ra_a'], associated_systems_table[rownum]['ra_b']
#     dec_a, dec_b = associated_systems_table[rownum]['dec_a'], associated_systems_table[rownum]['dec_b'] 
#     # use the ra and dec of component a and b to make SkyCoord objects
#     coord_a = SkyCoord(ra_a, dec_a, frame='icrs')
#     coord_b = SkyCoord(ra_b, dec_b, frame='icrs')

#     # calcualte the separation between the two objects
#     separation = coord_a.separation(coord_b)
#     associated_systems_table['separation'][rownum] = separation
    
#     if rownum%5000==0:
#         print(rownum)
    

non_associated_systems_table.add_column(0*u.arcsec, name='separation')
non_associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu_orbit')
non_associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_ra2')
non_associated_systems_table.add_column(0*u.mas**2/u.yr**2, name='delta_mu_dec2')
non_associated_systems_table.add_column(0*u.mas/u.yr, name='delta_mu')
non_associated_systems_table.add_column(0*u.mas/u.yr, name='sigma_delta_mu')


for rownum in range(len(query_results_table)):
    ra_a, ra_b = non_associated_systems_table[rownum]['ra_a'], non_associated_systems_table[rownum]['ra_b']
    dec_a, dec_b = non_associated_systems_table[rownum]['dec_a'], non_associated_systems_table[rownum]['dec_b'] 
    # use the ra and dec of component a and b to make SkyCoord objects
    coord_a = SkyCoord(ra_a, dec_a, frame='icrs')
    coord_b = SkyCoord(ra_b, dec_b, frame='icrs')

    # calcualte the separation between the two objects
    separation = coord_a.separation(coord_b)
    non_associated_systems_table['separation'][rownum] = separation
    if rownum%5000==0:
      print(rownum)
    
## Remove the rows from the associated_systems_table
table_length = len(associated_systems_table)
associated_systems_table.remove_rows(slice(0,table_length))

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [20]:
associated_systems_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcsec,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


## LOOP THROUGH RESULTS TABLE AND RECORD WHICH ROWS ARE NON-ASSOCIATED PAIRS

In [ ]:
# test

rownum = 0

""" READ IN THE RELEVANT VALUES:  ra, dec, parallax, pmra, pmdec, g magnitude, parallax uncertainties, proper motion uncertainties """
ra_a, ra_b = query_results_table[rownum]['ra_a'], query_results_table[rownum]['ra_b']
dec_a, dec_b = query_results_table[rownum]['dec_a'], query_results_table[rownum]['dec_b']
parallax_a, parallax_b = query_results_table[rownum]['parallax_a'], query_results_table[rownum]['parallax_b']
pmra_a, pmra_b = query_results_table[rownum]['pmra_a'], query_results_table[rownum]['pmra_b']
pmdec_a, pmdec_b = query_results_table[rownum]['pmdec_a'], query_results_table[rownum]['pmdec_a']
phot_g_mean_mag_a, phot_g_mean_mag_b = query_results_table[rownum]['phot_g_mean_mag_a'], query_results_table[rownum]['phot_g_mean_mag_b']
parallax_error_a, parallax_error_b = query_results_table[rownum]['parallax_error_a'], query_results_table[rownum]['parallax_error_b']
pmra_error_a, pmra_error_b = query_results_table[rownum]['pmra_error_a'], query_results_table[rownum]['pmra_error_b']
pmdec_error_a, pmdec_error_b = query_results_table[rownum]['pmdec_error_a'], query_results_table[rownum]['pmdec_error_a']
separation = query_results_table[rownum]['separation']


""" ASTROMETRIC CUT 1) PROJECTED SEPARATIONS """


# figure out which component has brighter g mag and indicate brighter object's parallax
# if one of the mags is missing, assume component a is brighter
if phot_g_mean_mag_a > phot_g_mean_mag_b: # larger mag -> dimmer object
    brighter_component_parallax = parallax_b
else:
    brighter_component_parallax = parallax_a
    
# if the separation is too large, then the pair is not associated
if separation > 206.25*brighter_component_parallax:
    non_associated_rownum.append(rownum)
    
    

    

In [ ]:
len(non_associated_systems_table)

0

In [15]:
""" INITIALIZE non_associated_rownum LIST TO KEEP TRACK OF OBJECTS THAT DO NOT PASS ANY ONE OF THE CUTS """

non_associated_rownum = []
associated_systems_rownum = []



""" Go through query results table and sort objects as associated or non-associated """
for rownum in range(len(query_results_table)):

    if rownum%5000==0:
        print(rownum)

    """ READ IN THE RELEVANT VALUES:  ra, dec, parallax, pmra, pmdec, g magnitude, parallax uncertainties, proper motion uncertainties """
    ra_a, ra_b = query_results_table[rownum]['ra_a'], query_results_table[rownum]['ra_b']
    dec_a, dec_b = query_results_table[rownum]['dec_a'], query_results_table[rownum]['dec_b']
    parallax_a, parallax_b = query_results_table[rownum]['parallax_a'], query_results_table[rownum]['parallax_b']
    pmra_a, pmra_b = query_results_table[rownum]['pmra_a'], query_results_table[rownum]['pmra_b']
    pmdec_a, pmdec_b = query_results_table[rownum]['pmdec_a'], query_results_table[rownum]['pmdec_b']
    phot_g_mean_mag_a, phot_g_mean_mag_b = query_results_table[rownum]['phot_g_mean_mag_a'], query_results_table[rownum]['phot_g_mean_mag_b']
    parallax_error_a, parallax_error_b = query_results_table[rownum]['parallax_error_a'], query_results_table[rownum]['parallax_error_b']
    pmra_error_a, pmra_error_b = query_results_table[rownum]['pmra_error_a'], query_results_table[rownum]['pmra_error_b']
    pmdec_error_a, pmdec_error_b = query_results_table[rownum]['pmdec_error_a'], query_results_table[rownum]['pmdec_error_a']
    separation = query_results_table[rownum]['separation']

    # proper motion calculations:
    
    # figure out which component has brighter g mag and indicate brighter object's parallax
    # if one of the mags is missing, assume component a is brighter
    if phot_g_mean_mag_a > phot_g_mean_mag_b: # larger mag -> dimmer object
        brighter_component_parallax = parallax_b
    else:
        brighter_component_parallax = parallax_a
    
    # delta_mu_orbit is the max proper motion difference expected for orbital motion (depending on given sep and parallax)
    delta_mu_orbit = ( 0.44*(brighter_component_parallax/u.mas)**(3/2) * (separation.to(u.arcsec)/u.arcsec)**(1/2) )*u.mas/u.yr
    
    # mu_star is pmra * cos(dec)
    # note: the error in ra is really the error of this value with cosine included
    # another note: can't do cos of units
    mu_star_ra_a = pmra_a/u.mas * cos(dec_a/u.mas)
    mu_star_ra_b = pmra_b/u.mas * cos(dec_b/u.mas)
    mu_star_ra_a = mu_star_ra_a*u.mas
    mu_star_ra_b = mu_star_ra_b*u.mas
    
    # delta_mu_ra2 is the square of the difference of mu_star_a - mu_star_b
    delta_mu_ra2 = (mu_star_ra_a - mu_star_ra_b)**2
    
    # delta_mu_dec2 is the square of the difference of pmdec_a - pmdec_b
    delta_mu_dec2 = (pmdec_a - pmdec_b)**2
        
    # delta_mu is the scalar proper motion difference, which is:
    # [ (mu_star_ra_a - mu_star_ra_b)^2 +(pmdec_a - pmdec_b)^2 ]^(1/2)
    delta_mu = ( (mu_star_ra_a - mu_star_ra_b)**2 + (pmdec_a - pmdec_b)**2 )**(1/2)
    

    # sigma_delta_mu is (I believe) the weighted uncertainty of delta_mu
    sigma_delta_mu = (1/delta_mu) * ( (pmra_error_a**2 + pmra_error_b**2) * delta_mu_ra2 + (pmdec_error_a**2 + pmdec_error_b**2) * delta_mu_dec2 )**(1/2)
    
    
    # update these pm calculations in the query results table and associated systems table
    query_results_table['delta_mu_orbit'][rownum] = delta_mu_orbit
    query_results_table['delta_mu_ra2'][rownum] = delta_mu_ra2
    query_results_table['delta_mu_dec2'][rownum] = delta_mu_dec2
    query_results_table['delta_mu'][rownum] = delta_mu
    query_results_table['sigma_delta_mu'][rownum] = sigma_delta_mu
    
    
    non_associated_systems_table['delta_mu_orbit'][rownum] = delta_mu_orbit
    non_associated_systems_table['delta_mu_ra2'][rownum] = delta_mu_ra2
    non_associated_systems_table['delta_mu_dec2'][rownum] = delta_mu_dec2
    non_associated_systems_table['delta_mu'][rownum] = delta_mu
    non_associated_systems_table['sigma_delta_mu'][rownum] = sigma_delta_mu
    
    
    
    """ ASTROMETRIC CUT 1) PROJECTED SEPARATIONS """
    # if the 2-d separation in the sky is too large, the pair is not physically associated
    # so, we should add this row number to the list of rows that are not associated, and then move on to the next rownum
    if separation/u.arcsec > 206.25*brighter_component_parallax/u.mas:
        non_associated_rownum.append(rownum)
        continue
        
    # if the separation is close enough, we move on to the next round of cuts
    
    """ ASTROMETRIC CUT 2) PARALLAX SIMILARITY """
    # If the absolute value of the differnce of the parallax is too large, then the 3-d separation is too large,
    # therefore, the pair is not associated
    # This cut-off depends on the separation
    
    # absolute value of the difference of the parallaxes
    parallax_diff = abs(parallax_a - parallax_b)
    
    # add the uncertainties of the parallaxes in quadrature
    # note that there can't be units in the square root
    parallax_error_quad = sqrt((parallax_error_a**2 + parallax_error_b**2)/u.mas**2)
    parallax_error_quad = parallax_error_quad*u.mas
    
    # set the value of b depending on the separation
    if separation > 4*u.arcsec:
        b = 3
    elif separation <= 4*u.arcsec:
        b = 6
        
    # if the parallax diff is too large, the pair is not associated
    # so, we should add this row number to the list of rows that are not associated, and then move on to the next rownum
    if parallax_diff > b * parallax_error_quad:
        non_associated_rownum.append(rownum)
        continue
    
    """ ASTROMETRIC CUT 3) PROPER MOTION SIMILARITY """
    # now if the proper motions aren't sufficiently similar, those pairs are not associated


    
    # if the pm difference is too large, the pair isn't physically associated
    if delta_mu >= delta_mu_orbit + 2 * sigma_delta_mu:
        non_associated_rownum.append(rownum)
        continue
    
    else:   
      associated_systems_rownum.append(rownum) 
 

0
5000


/usr/local/lib/python3.9/dist-packages/astropy/units/quantity.py:673: RuntimeWarning: divide by zero encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/usr/local/lib/python3.9/dist-packages/astropy/units/quantity.py:673: RuntimeWarning: invalid value encountered in multiply
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [16]:
len(associated_systems_rownum)

36357

In [ ]:
# finalize the rows in associated and non-associated tables     
        
non_associated_systems_table_rownum = 0
        
for rownum in reversed(non_associated_rownum):
    
    if rownum%5000==0:
        print(rownum)
    
    associated_systems_table.remove_row(rownum)
    non_associated_systems_table.add_row()
    non_associated_systems_table[non_associated_systems_table_rownum] = query_results_table[rownum]
    non_associated_systems_table_rownum +=1
    


105000
100000
95000
90000
85000
80000
75000
70000
65000
60000
50000
45000
40000
30000
25000
20000
15000
10000
5000
0


In [ ]:
## vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# wiser's lab comp:
save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

# write the output files to end in _c# where # is the core number that was used

ascii.write(associated_systems_table, '{path}/associated_systems_table.ecsv'.format(path = save_path), format='ecsv',overwrite=True)
ascii.write(associated_systems_table, '{path}/associated_systems_table.csv'.format(path = save_path), format='csv',overwrite=True)

ascii.write(non_associated_systems_table, '{path}/non_associated_systems_table.ecsv'.format(path = save_path), format='ecsv',overwrite=True)
ascii.write(non_associated_systems_table, '{path}/non_associated_systems_table.csv'.format(path = save_path), format='csv',overwrite=True)

In [ ]:
associated_systems_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mas,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292,6593.5287484106,5.84929452335804,65.37566605693283,0.029512516295242767,8.08734681915077,0.032454123037433946
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429,8133.223121880968,22.71564110834343,1.178308274619283,0.05300534921025606,1.1096457199617988,0.0686100192383481
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486

In [ ]:
non_associated_systems_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mas,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
23599-3112TDT,154512,2434946053936490112,2434946053936490624,2016.0,2016.0,359.96355500588083,359.96262455196177,0.032081302255392075,0.017686752602458,-9.612379299063203,-9.61286123239272,0.03117678500711918,0.01464757975190878,2.179866400428931,1.9761075142725293,0.048033613711595535,0.023070620372891426,45.382102966308594,85.65471649169922,18.45443485643451,18.46177236503135,0.03933011367917061,0.021227160468697548,-3.319115917546042,-3.405950197123499,0.034288618713617325,0.015910563990473747,1.3019764423370361,-4.182226657867432,2.7199864387512207,2.7394676208496094,31.0,31.0,13.0,14.0,0.06435806304216385,0.03107656165957451,1.9168193340301514,0.9846497178077698,11.698969841003418,13.467615127563477,3699.547607421875,4054.001708984375,11.975320816040039,13.753835678100586,1682.1719970703125,245.90370178222656,11.253193855285645,12.956023216247559,2170.3193359375,446.86810302734375,0.7221269607543945,0.7978124618530273,1.1968821287155151,1.2314550876617432,3730.585963194251,2.7351806694944614,195.97198773844462,0.007540192109736011,13.999268835569747,0.044692995002613464
23599-0937KPP,154511,2434946053936490112,2434946053936490624,2016.0,2016.0,359.96355500588083,359.96262455196177,0.032081302255392075,0.017686752602458,-9.612379299063203,-9.61286123239272,0.03117678500711918,0.01464757975190878,2.179866400428931,1.9761075142725293,0.048033613711595535,0.023070620372891426,45.382102966308594,85.65471649169922,18.45443485643451,18.46177236503135,0.03933011367917061,0.021227160468697548,-3.319115917546042,-3.405950197123499,0.034288618713617325,0.015910563990473747,1.3019764423370361,-4.182226657867432,2.7199864387512207,2.7394676208496094,31.0,31.0,13.0,14.0,0.06435806304216385,0.03107656165957451,1.9168193340301514,0.9846497178077698,11.698969841003418,13.467615127563477,3699.547607421875,4054.001708984375,11.975320816040039,13.753835678100586,1682.1719970703125,245.90370178222656,11.253193855285645,12.956023216247559,2170.3193359375,446.86810302734375,0.7221269607543945,0.7978124618530273,1.1968821287155151,1.2314550876617432,3730.585963194251,2.7351806694944614,195.97198773844462,0.007540192109736011,13.999268835569747,0.044692995002613464
23599-0412CBL,154510,2447741345626066816,2447741414345542912,2016.0,2016.0,359.9891027371329

In [ ]:
query_results_table

wds_identifier,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b,separation,delta_mu_orbit,delta_mu_ra2,delta_mu_dec2,delta_mu,sigma_delta_mu
,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,,mas,mas / yr,mas2 / yr2,mas2 / yr2,mas / yr,mas / yr
str13,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292,6593.5287484106,5.84929452335804,65.37566605693283,0.029512516295242767,8.08734681915077,0.032454123037433946
00001+7727LOC,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429,8133.223121880968,6.49644671672915,1.178308274619283,0.05300534921025606,1.1096457199617988,0.0686100192383481
00001+5400ES,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486314701,0.0083489781245589